In [76]:
import pandas as pd 
from sqlalchemy import create_engine
server = 'rice-sqlcl-db'
database = 'SharadarEquity'
username = 'stocks'
password = '6LAZH1'
string = f"mssql+pymssql://{username}:{password}@{server}/{database}"
conn = create_engine(string).connect() 

In [77]:
prices = pd.read_csv("../sep.csv")
prices = prices[prices.date>="2024-04-01"][["ticker", "date", "close"]]
prices = prices.sort_values(by=["ticker", "date"])
prices = prices.groupby("ticker").last()

In [78]:
financials="eps roe roa ros grossmargin netmargin ebitdamargin assetturnover payoutratio de currentratio assets debt equity revenue".split()

In [83]:
df = pd.read_sql("select * from fundamentals where dimension='ARY' and reportperiod>='2023-01-01' order by ticker, reportperiod", conn)
df.set_index(["ticker", "reportperiod"], inplace=True)
df = df[df.netinccmn>0]
df['eps_gr'] = df.groupby("ticker").eps.pct_change() 
df['asset_gr'] = df.groupby("ticker").assets.pct_change() 
df['revenue_gr'] = df.groupby("ticker").revenue.pct_change() 
df = df.groupby("ticker").last()
df["netinc_grprof"] = df.netinccmn / df.gp 
df = df[financials+["eps_gr", "asset_gr", "revenue_gr", "netinc_grprof"]]
df = df.rename(columns={"ros": "ebitmargin"})
df = df.merge(prices["close"], left_index=True, right_index=True, how="inner")
df = df.dropna()
df["lvg"] = df.assets / df.equity
df["pe"] = df.close / df.eps
df = df.drop(columns=["eps", "close"])

C:\Users\kerry\AppData\Local\Temp\ipykernel_33116\914411082.py:4: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['eps_gr'] = df.groupby("ticker").eps.pct_change()


In [84]:
tickers = pd.read_sql("select * from tickers", conn, index_col="ticker")
tickers = tickers["name exchange sector scalerevenue".split()]
df = tickers.merge(df, left_index=True, right_index=True, how="inner")
df = df.dropna()
df.to_excel("financials.xlsx")

In [85]:
df.shape

(1359, 24)